Importing Required Libraries :

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Reading And Preprocessing The Dataset :

In [2]:
#reading ds
ds = pd.read_csv('spam.csv', encoding="ISO-8859-1")
ds.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
#preprocessing ds
ds.info() #checking datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [4]:
X = ds.v2
Y = ds.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [6]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

Creating Model And Adding Layers:

In [7]:
#adding layers in model
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

Compiling The Model:

In [8]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Fit The Model:

In [9]:
model.fit(sequences_matrix,Y_train,
          batch_size=128,
          epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 28s 486ms/step - loss: 0.3243 - accuracy: 0.8743 - val_loss: 0.1668 - val_accuracy: 0.9473
Epoch 2/10
30/30 [==============================] - 14s 465ms/step - loss: 0.0730 - accuracy: 0.9810 - val_loss: 0.0759 - val_accuracy: 0.9789
Epoch 3/10
30/30 [==============================] - 14s 458ms/step - loss: 0.0390 - accuracy: 0.9902 - val_loss: 0.0908 - val_accuracy: 0.9747
Epoch 4/10
30/30 [==============================] - 15s 484ms/step - loss: 0.0270 - accuracy: 0.9918 - val_loss: 0.0646 - val_accuracy: 0.9810
Epoch 5/10
30/30 [==============================] - 15s 498ms/step - loss: 0.0206 - accuracy: 0.9939 - val_loss: 0.0654 - val_accuracy: 0.9800
Epoch 6/10
30/30 [==============================] - 16s 525ms/step - loss: 0.0164 - accuracy: 0.9955 - val_loss: 0.0671 - val_accuracy: 0.9810
Epoch 7/10
30/30 [==============================] - 16s 518ms/step - loss: 0.0110 - accuracy: 0.9966 - val_loss: 0.0678 - val_accuracy: 0.9831

Saving The Model:

In [10]:
model.save('sms_spam_classifier.h5')

Testing The Model:

In [11]:
#preprocessing test ds
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

#testing
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 38ms/step - loss: 0.1760 - accuracy: 0.9749


In [12]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.176
  Accuracy: 0.975
